## Testing PySpark Connectivity

In [ ]:
%%local
print("Demo Notebook")

### Connection to EMR Cluster

In the cell below, the code block is autogenerated. You can generate this code by clicking on the "Cluster" link on the top of the notebook and select the EMR cluster. The "j-xxxxxxxxxxxx" is the cluster id of the cluster selected. 

For the example in this [blog](https://aws.amazon.com/blogs/machine-learning/part-1-create-and-manage-amazon-emr-clusters-from-sagemaker-studio-to-run-interactive-spark-and-ml-workloads/), we used a no-auth cluster for simplicity, but this works equally well for Kerberos, LDAP and HTTP auth mechanisms

In [ ]:
# %load_ext sagemaker_studio_analytics_extension.magics
# %sm_analytics emr connect --verify-certificate False --cluster-id <> --auth-type None --language python  

In [ ]:
%%info

In the next cell, we will use the HiveContext to query Hive and look at the databases and tables

In [ ]:
dbs = sqlContext.sql("show databases")
dbs.show()

tables = sqlContext.sql("show tables")
tables.show()

In [ ]:
movie_reviews = sqlContext.sql("select * from movie_reviews").cache()

In [ ]:
# Shape
print((movie_reviews.count(), len(movie_reviews.columns)))
# count of both positive and negative sentiments
movie_reviews.groupBy('sentiment').count().show()

In [ ]:
pos_reviews = movie_reviews.filter(movie_reviews.sentiment == 'positive').collect()
neg_reviews = movie_reviews.filter(movie_reviews.sentiment == 'negative').collect()

In [ ]:
import matplotlib.pyplot as plt
def plot_counts(positive,negative):
    plt.rcParams['figure.figsize']=(6,6)
    plt.bar(0,positive,width=0.6,label='Positive Reviews',color='Green')
    plt.bar(2,negative,width=0.6,label='Negative Reviews',color='Red')
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.ylabel('Count')
    plt.xlabel('Type of Review')
    plt.tick_params(
        axis='x',          
        which='both',      
        bottom=False,      
        top=False,         
        labelbottom=False) 
    plt.show()
    
plot_counts(len(pos_reviews),len(neg_reviews))
%matplot plt

Next, Let's inspect length of reviews using the pyspark.sql.functions module

In [ ]:
from pyspark.sql.functions import length
reviewlengthDF = movie_reviews.select(length('review').alias('Length of Review')) 
reviewlengthDF.show() 